This notebook is used to clean the data and get it into a relational table format.

In [2]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

# Unpack the nested map structure to the needed user data in a table

In [3]:
users_packed = pd.read_json('../data/user_ticks.json', lines=True)

In [4]:
users_packed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7350 entries, 0 to 7349
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   _id      7350 non-null   object
 1   user_id  7350 non-null   int64 
 2   ticks    7350 non-null   object
dtypes: int64(1), object(2)
memory usage: 172.4+ KB


In [ ]:
users = pd.DataFrame(users_packed.loc[0,['ticks']][0])
users['user_id'] = users_packed.loc[0,['user_id']][0]

for idx in range(1, users_packed.count()[0]):
    new_df = pd.DataFrame(users_packed.loc[idx,['ticks']][0])
    new_df['user_id'] = users_packed.loc[idx,['user_id']][0]
    users = users.append(new_df, ignore_index=True)

In [65]:
users = users.rename(columns={
    'routeId': 'route_id', 
    'leadStyle': 'lead_style', 
    'tickId': 'tick_id', 
    'userStars': 'user_stars', 
    'userRating': 'user_rating'})
# users['date_time'] = pd.to_datetime(users['date'])
users = users.astype({'route_id': 'int32', 'tick_id': 'int32'})
for col in ['date','notes','style','lead_style','user_rating']:
    users[col] = users[col].apply(lambda x: None if x == '' else x)
users['user_stars'] = users['user_stars'].apply(lambda x: np.nan if x == -1.0 else x)
users['notes'] = users['notes'].apply(lambda x: x.replace('\n','') if isinstance(x, str) else x)
users['notes'] = users['notes'].apply(lambda x: x.replace('\r','') if isinstance(x, str) else x)
users['notes'] = users['notes'].apply(lambda x: x.replace('\t','') if isinstance(x, str) else x)

In [66]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576208 entries, 0 to 576207
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   route_id     576208 non-null  int32  
 1   date         576208 non-null  object 
 2   pitches      576208 non-null  float64
 3   notes        227605 non-null  object 
 4   style        346484 non-null  object 
 5   lead_style   211575 non-null  object 
 6   tick_id      576208 non-null  int32  
 7   user_stars   369223 non-null  float64
 8   user_rating  115941 non-null  object 
 9   user_id      576208 non-null  int64  
dtypes: float64(2), int32(2), int64(1), object(5)
memory usage: 39.6+ MB


In [69]:
users

,route_id,date,pitches,notes,style,lead_style,tick_id,user_stars,user_rating,user_id
0,106002151,2019-11-02,1.0,None,TR,None,118003759,NaN,None,200527767
1,106241707,2019-11-02,1.0,None,TR,None,118003758,NaN,None,200527767
2,106589615,2019-11-02,1.0,None,Lead,None,118003757,NaN,None,200527767
3,106002147,2019-11-02,1.0,None,Lead,None,117978517,NaN,None,200527767
4,106810828,2019-11-02,1.0,None,TR,None,117978240,NaN,None,200527767
...,...,...,...,...,...,...,...,...,...,...
576203,107723683,2017-12-30,1.0,onsight,None,None,113971775,NaN,None,106930862
576204,106224432,2017-12-30,2.0,onsight,None,None,113971766,3.0,None,106930862
576205,106224397,2017-12-30,1.0,flash,None,None,113971764,2.0,None,106930862
576206,105895574,2017-12-22,1.0,onsight,None,None,113948901,NaN,None,106930862


In [68]:
users.to_csv('../data/user_ticks.csv', header=True, index=False, sep='|')

# Convert routes to a table and add numerical rating

In [16]:
routes = pd.read_json('../data/routes.json', lines=True)

In [17]:
routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80495 entries, 0 to 80494
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   _id          80495 non-null  object 
 1   id           80495 non-null  int64  
 2   name         80495 non-null  object 
 3   type         80495 non-null  object 
 4   rating       80495 non-null  object 
 5   stars        80495 non-null  float64
 6   starVotes    80495 non-null  int64  
 7   pitches      80495 non-null  object 
 8   location     80495 non-null  object 
 9   url          80495 non-null  object 
 10  imgSqSmall   80495 non-null  object 
 11  imgSmall     80495 non-null  object 
 12  imgSmallMed  80495 non-null  object 
 13  imgMedium    80495 non-null  object 
 14  longitude    80495 non-null  float64
 15  latitude     80495 non-null  float64
dtypes: float64(3), int64(2), object(11)
memory usage: 9.8+ MB


In [18]:
routes = routes.drop(columns=['_id','imgSqSmall','imgSmall','imgSmallMed','imgMedium', 'url'])
routes = routes.drop_duplicates(subset='id',ignore_index=True)
routes = routes.rename(columns={'id': 'route_id', 'starVotes': 'star_votes'})
routes['pitches'] = routes['pitches'].apply(lambda x: int(x) if x else 1.0)

In [19]:
routes['base_rating'] = routes['rating']
routes['base_rating'] = routes['base_rating'].apply(lambda x: x.split()[0] if len(x.split()) > 0 else '')
routes['stars'] = routes['stars'].apply(lambda x: x-1)

A complex rating conversion scheme was developed to be able to look at routes with a numerical rating for statistics comparisons. Only routes with a "5.x" or "Vx" rating were considered.

In [20]:
rating_conv = {
    '5.0': 0.0,
    '5.1': 1.0,
    '5.2': 2.0,
    '5.3': 3.0,
    '5.4': 4.0,
    '5.5': 5.0,
    '5.6': 6.0,
    '5.7-': 7.0,
    '5.7': 7.3,
    '5.7+': 7.7,
    '5.8-': 8.0,
    '5.8': 8.3,
    '5.8+': 8.7,
    '5.9-': 9.0,
    '5.9': 9.3,
    '5.9+': 9.7,
    '5.10-': 10.0,
    '5.10a': 10.0,
    '5.10': 10.45,
    '5.10b': 10.3,
    '5.10c': 10.6,
    '5.10d': 10.8,
    '5.10+': 10.8,
    '5.11-': 11.0,
    '5.11a': 11.0,
    '5.11': 11.45,
    '5.11b': 11.3,
    '5.11c': 11.6,
    '5.11d': 11.8,
    '5.11+': 11.8,
    '5.12-': 12.0,
    '5.12a': 12.0,
    '5.12': 12.45,
    '5.12b': 12.3,
    '5.12c': 12.6,
    '5.12d': 12.8,
    '5.12+': 12.8,
    '5.13-': 13.0,
    '5.13a': 13.0,
    '5.13': 13.45,
    '5.13b': 13.3,
    '5.13c': 13.6,
    '5.13d': 13.8,
    '5.13+': 13.8,
    '5.14-': 14.0,
    '5.14a': 14.0,
    '5.14': 14.45,
    '5.14b': 14.3,
    '5.14c': 14.6,
    '5.14d': 14.8,
    '5.14+': 14.8,
    '5.15-': 15.0,
    '5.15a': 15.0,
    '5.15': 15.45,
    '5.15b': 15.3,
    '5.15c': 15.6,
    '5.15d': 15.8,
    '5.15+': 15.8,
    '5.10a/b': 10.15,
    '5.10b/c': 10.45,
    '5.10c/d': 10.7,
    '5.11a/b': 11.15,
    '5.11b/c': 11.45,
    '5.11c/d': 11.7,
    '5.12a/b': 12.15,
    '5.12b/c': 12.45,
    '5.12c/d': 12.7,
    '5.13a/b': 13.15,
    '5.13b/c': 13.45,
    '5.13c/d': 13.7,
    '5.14a/b': 14.15,
    '5.14b/c': 14.45,
    '5.14c/d': 14.7,
    '5.15a/b': 15.15,
    '5.15b/c': 15.45,
    '5.15c/d': 15.7,
    'V0': 10.8,
    'V1': 11.0,
    'V2': 11.3,
    'V3': 11.6,
    'V4': 11.8,
    'V5': 12.0,
    'V6': 12.3,
    'V7': 12.6,
    'V8': 12.8,
    'V9': 13.0,
    'V10': 13.3,
    'V11': 13.6,
    'V12': 13.8,
    'V0-V1': 10.9,
    'V1-V2': 11.15,
    'V2-V3': 11.45,
    'V3-V4': 11.7,
    'V4-V5': 11.9,
    'V5-V6': 12.15,
}

json.dump(rating_conv, open("rating_conv.json", 'w' ))

In [21]:
routes['num_rating'] = routes['base_rating'].apply(lambda x: rating_conv[x] if x in rating_conv.keys() else np.nan)

In [22]:
routes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76202 entries, 0 to 76201
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   route_id     76202 non-null  int64  
 1   name         76202 non-null  object 
 2   type         76202 non-null  object 
 3   rating       76202 non-null  object 
 4   stars        76202 non-null  float64
 5   star_votes   76202 non-null  int64  
 6   pitches      76202 non-null  float64
 7   location     76202 non-null  object 
 8   longitude    76202 non-null  float64
 9   latitude     76202 non-null  float64
 10  base_rating  76202 non-null  object 
 11  num_rating   71121 non-null  float64
dtypes: float64(5), int64(2), object(5)
memory usage: 7.0+ MB


In [23]:
routes

,route_id,name,type,rating,stars,star_votes,pitches,location,longitude,latitude,base_rating,num_rating
0,105806397,The Grand Wall,"Trad, Aid",5.11a A0,3.9,324,9.0,"[International, North America, Canada, British...",-123.1480,49.6822,5.11a,11.00
1,105806955,Exasperator,Trad,5.10c,4.0,450,2.0,"[International, North America, Canada, British...",-123.1481,49.6823,5.10c,10.60
2,105947052,Klahanie Crack,Trad,5.7,3.7,465,1.0,"[International, North America, Canada, British...",-123.1578,49.6696,5.7,7.30
3,105842838,Crime of the Century,Trad,5.11b/c,3.8,229,1.0,"[International, North America, Canada, British...",-123.1378,49.7050,5.11b/c,11.45
4,107198282,Skywalker,Trad,5.8,3.5,473,5.0,"[International, North America, Canada, British...",-123.1583,49.6693,5.8,8.30
...,...,...,...,...,...,...,...,...,...,...,...,...
76197,106430462,Animal Acts,Boulder,V5,3.0,32,1.0,"[Texas, Hueco Tanks, West Mountain, Scream]",-106.0456,31.9129,V5,12.00
76198,106561511,Sunnyside Bench Regular Route,Trad,5.5,2.5,268,3.0,"[California, Yosemite National Park, Yosemite ...",-119.5949,37.7508,5.5,5.00
76199,107610087,Super Final,Sport,5.12b,3.5,25,1.0,"[Tennessee, Obed & Clear Creek, South Clear Cr...",-84.7083,36.0947,5.12b,12.30
76200,107741178,Minnowmaker,Sport,5.12a/b,3.7,11,1.0,"[Utah, Wasatch Range, Logan, China Wall]",-111.6422,41.7854,5.12a/b,12.15


In [24]:
routes.to_csv('../data/routes.csv', header=True, index=False)